##### Воробьёв Сергей, ВМК МГУ, 2021
Решение задачи https://www.kaggle.com/c/iml2020-bank

### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from hyperopt import hp

In [2]:
%pylab inline
plt.style.use('seaborn-dark')
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None) # pd.options.display.max_columns = None 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
plt.rc('font', size=14)

Populating the interactive namespace from numpy and matplotlib


In [3]:
from sklearn.metrics import accuracy_score, roc_auc_score, \
confusion_matrix, precision_score, recall_score, mean_squared_error, \
f1_score
from sklearn.preprocessing import StandardScaler 

### Data

In [4]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
print(train.shape, test.shape)
train.sample(5)

(112500, 11) (37500, 10)


,плохой_клиент,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья
68290,0,0.194534,53,1,0.446851,3000.0,8,0,2,0,1.0
37993,0,0.294455,31,0,0.195893,4430.0,7,0,0,0,0.0
110140,0,0.721329,38,0,0.828586,2000.0,6,0,1,0,0.0
6476,0,0.173605,45,0,0.129263,5600.0,5,0,0,0,1.0
77363,0,0.011049,75,0,316.000000,NaN,12,0,0,0,0.0


In [5]:
train.describe()

,плохой_клиент,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья
count,112500.000000,112500.000000,112500.000000,112500.000000,112500.000000,9.027400e+04,112500.000000,112500.000000,112500.000000,112500.000000,109549.000000
mean,0.067058,6.057904,52.292702,0.425538,350.965444,6.662380e+03,8.464267,0.271627,1.019680,0.245333,0.755662
std,0.250123,257.959993,14.765029,4.246085,1864.557746,1.540162e+04,5.149137,4.223498,1.130324,4.208686,1.113419
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029840,41.000000,0.000000,0.175069,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154015,52.000000,0.000000,0.366336,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559389,63.000000,0.000000,0.863569,8.250000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,326442.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,13.000000


In [6]:
y = train.pop('плохой_клиент')
train.fillna(train.mean(), inplace=True) ## было -11
test.fillna(train.mean(), inplace=True)


ss =StandardScaler()
train = ss.fit_transform(train)
test = ss.transform(test)

train.shape, y.shape

((112500, 10), (112500,))

### Training & Validation

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
from IPython.display import clear_output

In [9]:
def fit_predict(X, y, test, par): 
    d_train=lgb.Dataset(X, label=y)
    #train the model 
    clf=lgb.train(par, d_train, 200) #train the model on 100 epocs
    #prediction on the test set
    y_pred=clf.predict(test)
    clear_output()
    return clf, y_pred

In [10]:
#Best result
""" 
        precision:  0.047238855622089154
        recall   :  0.7717391304347826
        f-score  :  0.08902821316614419
        accuracy :  0.9354222222222223   
"""
best_params={
    'learning_rate': 0.025,
    'num_iterations': 59,
    #'is_unbalance': True,
    'boosting_type':'dart',
    'objective': 'binary',
    'metric':'binary_logloss',
    'max_depth': 8,
}

params={
    'num_iterations': 154,
    'learning_rate': 0.00755,
    'boosting_type':'gbdt',
    'objective': 'binary',
    'metric':'binary_logloss',
    'num_leaves': 35,
    'max_depth': 16,
    
}

clf, y_pred = fit_predict(X_train, y_train, X_test, params)

In [11]:
print("precision: ", precision_score(np.where(y_pred > 0.5, 1, 0), y_test))
print("recall   : ", recall_score(np.where(y_pred > 0.5, 1, 0), y_test))
print("f-score  : ", f1_score(np.where(y_pred > 0.5, 1, 0), y_test))
print("auc-roc  : ", roc_auc_score(np.where(y_pred > 0.5, 1, 0), y_test))
print("accuracy : ", accuracy_score(np.where(y_pred > 0.5, 1, 0), y_test))
print("\n")
print(confusion_matrix(y_test, np.where(y_pred > 0.5, 1, 0)))

precision:  0.01330671989354624
recall   :  1.0
f-score  :  0.026263952724885097
auc-roc  :  0.9670151245551601
accuracy :  0.9340888888888889


[[20997     0]
 [ 1483    20]]


### Solution for Kaggle

In [12]:
d_train=lgb.Dataset(train, label=y)
best_model=lgb.train(params, d_train, 200) #train the model on 100 epocs
clear_output()

In [13]:
a = best_model.predict(test)
pd.DataFrame({'id': np.arange(37500), 'a':a}).to_csv('solution.csv', index=False)